Import Packages

In [204]:
import pandas as pd
import numpy as np
import warnings

from datetime import datetime, timedelta
from IPython.display import display
# !pip install -U pandasql
from pandasql import sqldf
mysql = lambda q: sqldf(q, globals())

warnings.filterwarnings("ignore")

Read Country Data

In [205]:
country_data_processed = pd.read_csv("Country_Processed_Table.csv")#read input data
country_data_processed.head()

,Country,Year,Country_Key,Continent,Region,Capital,Currency,Birth_Rate_Crude_Per_1000,Population_Total_Count,Death_Rate_Crude_Per_1000,GNI_Per_Capita_USD,Labour_Force_Female_Percent,Labour_Force_Total_Count,Age_Dependency_Ratio_Percent_Working_Age,Age_First_Marriage_Male,Age_First_Marriage_Female,Age_Dependency_Ratio_Old,Age_Dependency_Ratio_Young,Completeness_Birth_Registration_Percent
0,Canada,2005,0,North America,North America,Ottawa,CAD,10.6,32243753,7.1,34810,46.637376,17821886,44.495093,28.7,26.98,18.954219,25.540874,100.0
1,Canada,2006,1,North America,North America,Ottawa,CAD,10.9,32571174,7.0,38510,46.875172,18030705,44.236994,28.6,26.60,19.138346,25.098648,100.0
2,Canada,2007,2,North America,North America,Ottawa,CAD,11.2,32889025,7.1,43090,47.036713,18379605,44.034776,28.7,26.98,19.353443,24.681333,100.0
3,Canada,2008,3,North America,North America,Ottawa,CAD,11.3,33247118,7.2,45650,46.974754,18667574,43.933903,28.7,26.98,19.620978,24.312924,100.0
4,Canada,2009,4,North America,North America,Ottawa,CAD,11.3,33628895,7.1,43230,47.200377,18799155,43.976753,28.7,26.98,19.966597,24.010156,100.0


Create Fact Table and Add Month Data

In [206]:
fact_table = pd.DataFrame()

month_data_processed = pd.read_csv("Month_Processed_Table.csv")#read input data
fact_table = month_data_processed.merge(country_data_processed, on="Year")
fact_table = fact_table.sort_values('Country')

fact_table.head()


,Month_Key,Name,Month_Num,Quarter,Year,Decade,Country,Country_Key,Continent,Region,...,Death_Rate_Crude_Per_1000,GNI_Per_Capita_USD,Labour_Force_Female_Percent,Labour_Force_Total_Count,Age_Dependency_Ratio_Percent_Working_Age,Age_First_Marriage_Male,Age_First_Marriage_Female,Age_Dependency_Ratio_Old,Age_Dependency_Ratio_Young,Completeness_Birth_Registration_Percent
1491,165,October,10,4,2018,21,Bolivia,109,South America,South America,...,6.465,3840,39.209263,133000453,47.947297,26.675,22.44,8.665518,39.281779,71.9
1572,174,July,7,3,2019,21,Bolivia,110,South America,South America,...,6.510,4050,39.491377,136202238,47.635083,26.675,22.44,8.932369,38.702714,68.0
357,39,April,4,2,2008,20,Bolivia,99,South America,South America,...,6.849,1940,37.966295,110890081,51.508237,26.675,22.44,7.503345,44.004892,68.0
996,110,March,3,1,2014,21,Bolivia,105,South America,South America,...,6.437,3620,37.975372,122770335,49.357239,26.675,22.44,7.883945,41.473294,68.0
159,17,June,6,2,2006,20,Bolivia,97,South America,South America,...,7.087,1380,35.879524,103776784,52.414299,26.675,22.44,7.419868,44.994431,68.0


Connect Tables

In [207]:
population_data_processed = pd.read_csv("Pop_Processed_Table.csv")#read input data
education_processed = pd.read_csv("Education_Processed_Table.csv")#read input data
health_processed = pd.read_csv("Health_Dimension.csv")#read input data
quality_life_processed = pd.read_csv("QualityOfLife_Dimension.csv")

fact_table = fact_table.merge(education_processed, left_on=['Country','Year'], right_on=['Country_Ref','Year_Ref'], how='left').drop(columns=["Country_Ref", "Year_Ref"], axis=1)
fact_table = fact_table.merge(population_data_processed, left_on=['Country','Year'], right_on=['Country_Ref','Year_Ref'], how='left' ).drop(columns=["Country_Ref", "Year_Ref"], axis=1)
fact_table = fact_table.merge(health_processed, left_on=['Country','Year'], right_on=['Country_Ref','Date'], how='left').drop(columns=["Country_Ref", "Date"], axis=1)
fact_table = fact_table.merge(quality_life_processed, left_on=['Country','Year'], right_on=['Country_Ref','Date'], how='left').drop(columns=["Country_Ref", "Date"], axis=1)


events_processed = pd.read_csv("Event_Pro_Table.csv")#read input data

query = '''
Select * 
FROM fact_table 
left join events_processed

on
fact_table.Country = events_processed.Country_Name
and fact_table.Month_Num >= events_processed.Start_Month
and fact_table.Month_Num <= events_processed.End_Month
and fact_table.Year >= events_processed.Start_Year
and fact_table.Year <= events_processed.End_Year
'''

fact_table = mysql(query)

fact_table['Event_Key'] = fact_table['Event_Key'].fillna(-1)
fact_table['Event_Key'] = fact_table['Event_Key'].astype(int)

fact_table["Fact_Key"] = np.arange(len(fact_table))

fact_table = pd.DataFrame(fact_table, columns=["Fact_Key","Month_Key", "Country_Key", "Population_Key", "Education_Key", "Health_Key", "QOL_Key", "Event_Key", "Country", "Year", "Name", "Month_Num"])

fact_table.tail(50)

,Fact_Key,Month_Key,Country_Key,Population_Key,Education_Key,Health_Key,QOL_Key,Event_Key,Country,Year,Name,Month_Num
1688,1688,175,30,30,142,142,142,-1,USA,2019,August,8
1689,1689,61,21,21,133,133,133,-1,USA,2010,February,2
1690,1690,130,26,26,138,138,138,-1,USA,2015,November,11
1691,1691,16,17,17,129,129,129,-1,USA,2006,May,5
1692,1692,18,17,17,129,129,129,-1,USA,2006,July,7
1693,1693,62,21,21,133,133,133,-1,USA,2010,March,3
1694,1694,129,26,26,138,138,138,-1,USA,2015,October,10
1695,1695,63,21,21,133,133,133,-1,USA,2010,April,4
1696,1696,15,17,17,129,129,129,-1,USA,2006,April,4
1697,1697,128,26,26,138,138,138,-1,USA,2015,September,9


Add Measures

In [208]:
population_human_development_raw = pd.read_csv("human_development_index_raw.csv")#read input data


countries = ["Canada", "United States", "Mexico", "Burundi", "Chad", "Niger", "Bolivia", "Philippines", "Indonesia"]

population_human_development_processed = population_human_development_raw[population_human_development_raw["Entity"].isin(countries)]
population_human_development_processed = population_human_development_processed[population_human_development_processed["Year"] >2004]
population_human_development_processed = population_human_development_processed.drop(["Code"], axis=1)

country = "Canada"
for country in countries:
    for i in range(3):
        country_data = {}
        country_data["Entity"] = country
        country_data["Year"] = 2018+i
        population_human_development_processed = population_human_development_processed.append(pd.DataFrame([country_data]), ignore_index=True)


population_human_development_processed['Entity'].replace({"United States": "USA"}, inplace = True)
population_human_development_processed = population_human_development_processed.sort_values(["Entity","Year"], ascending=[True,True])
population_human_development_processed.fillna(5, inplace=True)#perhaps change to extrapolation

fact_table = fact_table.merge(population_human_development_processed, left_on=['Country','Year' ], right_on=['Entity','Year' ], how='left').drop(columns=["Entity"])
fact_table = fact_table.sort_values(["Country","Year"], ascending=[True,True])
display(fact_table.tail(100))
print(fact_table.shape)

,Fact_Key,Month_Key,Country_Key,Population_Key,Education_Key,Health_Key,QOL_Key,Event_Key,Country,Year,Name,Month_Num,Human Development Index (UNDP)
1698,1698,94,23,23,135,135,135,-1,USA,2012,November,11,0.918
1714,1714,90,23,23,135,135,135,-1,USA,2012,July,7,0.918
1737,1737,86,23,23,135,135,135,-1,USA,2012,March,3,0.918
1556,1556,97,24,24,136,136,136,-1,USA,2013,February,2,0.916
1575,1575,105,24,24,136,136,136,-1,USA,2013,October,10,0.916
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1591,1591,187,31,31,143,143,143,-1,USA,2020,August,8,5.000
1593,1593,183,31,31,143,143,143,-1,USA,2020,April,4,5.000
1600,1600,184,31,31,143,143,143,-1,USA,2020,May,5,5.000
1605,1605,185,31,31,143,143,143,-1,USA,2020,June,6,5.000


(1738, 13)


Drop unwanted columns and output to csv for db loading